***Méthodes numériques: Module 2***
***********************************
Space and Time: Modélisation de l'écoulement du trafic.
==
## Introduction #

Dans ce module nous allons résoudre une équation non-linéaire à deux variables en discrétisant à la fois dans le temps (en utilisant la forward-difference) et dans l'espace (en utilisant la backward-difference). Nous utiliserons également les nouveaux outils introduit dans ce module: sympy et lambdify, qui nous permettrons d'écrire des formules mathématiques dans nos codes. Nous importons donc comme dans le module 1 la librairie numpy et matplotlib pour pouvoir dessiner des graphiques mais également sympy et lambdify.

In [7]:
import numpy
import sympy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16
from sympy import init_printing
init_printing()
from sympy.utilities.lambdify import lambdify

## Principe #

L'équation que l'on se propose de résoudre est la suivante: $$\frac{\partial{\rho}}{\partial{t}}+\frac{\partial{F}}{\partial{\rho}}\frac{\partial{\rho}}{\partial{x}}=0$$ 

Cette équation modélise le flux du trafic de voitures sur l'autoroute. Il s'agit en fait de l'équation non-linéaire de convection appliquée à la densité de voiture, $\rho$. La fonction F représente le flux des voitures donné par $F=V \rho$ où V est la vitesse moyenne des voitures.

On modélise le fait que la vitesse varie en fonction de la densité de voiture en exprimant que la vitesse est maximale quand la densité de voiture est nulle et qu'elle est nulle quand la densité de voiture atteind une densité critique que l'on appelle $\rho_{max}$, ce qui se traduit par la formule suivante: 

$$V = V_{max}(1-\frac{\rho}{\rho_{max}})$$

Ainsi, on peut calculer que 
$$\frac{\partial{F}}{\partial{\rho}}=\frac{\partial{V \rho}}{\partial{\rho}} = V + \rho \frac{\partial{V}}{\partial{\rho}}$$

avec $$\frac{\partial{V}}{\partial{\rho}}=-\frac{V_{max}}{\rho_{max}}$$

On obtient finalement que $$\frac{\partial{F}}{\partial{\rho}}=V_{max} - 2\rho \frac{V_{max}}{\rho_{max}}$$
C'est-à-dire que l'équation que nous devons résoudre pour $\rho$ est $$\frac{\partial{\rho}}{\partial{t}}+(c_{1}\rho + c_{2})\frac{\partial{\rho}}{\partial{x}}=0$$ où $$c_{1}=-2\frac{V_{max}}{\rho_{max}}$$ et $$c_{2}=V_{max}$$

Il s'agit en fait d'une combinaison des deux exemples du Module 2. Maintenant que nous avons notre équation, nous pouvons la discrétiser. 
Les exposants représentent les indices de temps et les indices en dessous sont les indices pour l'espace. La discrétisation en avant pour le temps et en arrière pour l'espace nous donne l'équation suivante:
$$\frac{\rho_{i}^{n+1} - \rho_{i}^{n}}{\Delta t} + (c_{1}\rho_{i}^{n} + c_{2})(\frac{\rho_{i}^{n} - \rho_{i}^{n-1}}{\Delta x})=0$$

Dans cette équation, la seule inconnue à chaque itération est $\rho_{i}^{n+1}$. 